<h1>The 8-puzzle</h1>

In [1]:
# Import any packages you need here
from math import sqrt
from queue import PriorityQueue
import numpy as np

#Now, define the class PuzzleNode:
class PuzzleNode:
    """
    Class PuzzleNode: Provides a structure for performing A* search for the n^2-1 puzzle
    """
    
    '''
    PuzzleNode Class attributes:
    -- dimensions: Dimension of the puzzle
    -- puzzle_config: The start configuration for the puzzle
    -- f_score: Cost function for the configuration
    -- g_score: Actual path cost to the present node
    -- parent_node: Pointer to the parent node
    -- pruned: Prevents repetition if another node with same state is present
    -- goal_config: Creates the goal state based on the dimensions of the puzzle.
    '''
    
    def __init__(self, puzzle_config, parent_node = None, f_score = 0, g_score = 0):
        self.dimensions = len(puzzle_config)
        self.puzzle_config = np.array(puzzle_config).reshape(self.dimensions, self.dimensions)
        self.f_score = f_score
        self.g_score = g_score
        self.parent_node = parent_node
        self.pruned = False
        self.goal_config = np.arange(self.dimensions**2).reshape(self.dimensions, self.dimensions)
    
    def __str__(self):
        '''
        Representing the state of the puzzle.
        '''
        return ('|'+
                '|\n|'
                .join( '|'
                      .join('{:{size}d}'
                            .format(
                                num, 
                                size=len(str(self.dimensions**2-1))) 
                            for num in lines) for lines in self.puzzle_config) + '|')
    
    def __lt__(self, other):
        '''
        Defining the less than operator for the Priority Queues
        '''
        return self.f_score < other.f_score
    
    def checkIfSolved(self):
        '''
        Returns True if the puzzle is solved.
        Returns False if the puzzle is not solved.
        '''
        return(np.array_equal(self.puzzle_config, self.goal_config))        

In [2]:
from memoization import cached

# Misplaced tiles heuristic
@cached
def h1(state):
    """
    This function returns the number of misplaced tiles, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the number of misplaced tiles
    """
    # Flattens the state
    state = np.array(state).flatten()
    
    # Calculates and returns the number of misplaced tiles from the goal state
    return(sum(i != j for i, j in zip(state, range(len(state)**2)))-1)

# Manhattan distance heuristic
@cached
def h2(state):
    """
    This function returns the Manhattan distance from the solved state, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Manhattan distance from the solved configuration
    """
    # Flattens the given list
    state = [item for sublist in state for item in sublist]
    
    # Dimensions of the state
    d = int(sqrt(len(state)))
    
    # Goal state
    goal = list(range(d**2))
    
    # Calculates the Manhattan distance
    # Sum of the distance for each tiles from the goal state to the current state
    return sum(abs(b%d - g%d) + abs(b//d - g//d) for b, g in ((state.index(i), goal.index(i)) for i in range(1, d**2)))
    
# Extra heuristic for the extension.  If implemented, modify the function below    
def h3(state):
    """
    This function returns a heuristic that dominates the Manhattan distance, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Heuristic distance of the state from its solved configuration
    """
    pass

    
# If you implement more than 3 heuristics, then add any extra heuristic functions onto the end of this list.
heuristics = [h1, h2]

In [3]:
# Import any packages or define any helper functions you need here

#Extension Question
# This is to find the total number of inversions for the initial state
def numInversions(st):
    st = [i for li in st for i in li]
    num_inv = 0
    # If the value of the tiles are in opposite order compared to that of the goal state.
    for i in range(8):
        for j in range(i, 9):
            if (st[i]>st[j] and st[j]!=0):
                num_inv += 1
    return num_inv

def testIfSolvable(state):
    '''
    For a puzzle with odd dimensions, the total number of inversions must be even for it to be solvable.
    Return true if the problem is solvable
    Return false if the problem is unsolvable
    '''
    if len(state)==3:
        num_inversions = numInversions(state)
        return (num_inversions%2 == 0)
    else:
        return True
        

# Main solvePuzzle function.
def solvePuzzle(state, heuristic):
    """This function should solve the n**2-1 puzzle for any n > 2 (although it may take too long for n > 4)).
    Inputs:
        -state: The initial state of the puzzle as a list of lists
        -heuristic: a handle to a heuristic function.  Will be one of those defined in Question 2.
    Outputs:
        -steps: The number of steps to optimally solve the puzzle (excluding the initial state)
        -exp: The number of nodes expanded to reach the solution
        -max_frontier: The maximum size of the frontier over the whole search
        -opt_path: The optimal path as a list of list of lists.  That is, opt_path[:,:,i] should give a list of lists
                    that represents the state of the board at the ith step of the solution.
        -err: An error code.  If state is not of the appropriate size and dimension, return -1.  For the extention task,
          if the state is not solvable, then return -2
    """
    # Dimensions of the given problem
    dim = len(state)
    
    # The three tests to check for dimensions and solvability.
    
    # Checks if the puzzle state is solvable.
    test_1 = testIfSolvable(state)
    
    # Checks if the length of each of the sublists is same as the dimension of the puzzle
    test_2 = set(map(len, state))=={dim}
    
    # Checks if the sorted order of initial state is same as the goal state.
    # For example: [[3,4,5],[0,1,2],[4,5,6]]'s sorted list will be [0,1,2,3,4,5,6,7,8]
    test_3 = sorted([i for li in state for i in li])== list(range(dim**2))
    
    # If any of these tests fail, we return the -1 error
    if not all([test_2, test_3]):
        return (0, 0, 0, None,-1)
    
    # If the solvability test fail, we return -2 error.
    if not test_1:
        return (0,0,0,None, -2)
    
    # This is the root node based on the initial state
    root = PuzzleNode(f_score = heuristic(state), puzzle_config =state)
    
    # Using numpy to get the goal state based on the dimensions
    goal_state = np.arange(dim**2).reshape((dim,dim))
    
    # We cannot hash the lists, so convert it to tuples.
    # Holds the costs to reach each of the already visisted nodes
    dict_costs = {
        tuple(root.puzzle_config.flatten()): root
    }
    
    # Implementation for the priority queue
    frontier = PriorityQueue()
    # Add the root node to the queue
    frontier.put(root)
    
    # Returns the number of nodes expanded during the search.
    exp = 0
    
    # Returns the number of steps to reach the goal state from a optimal solution.
    steps = 0
    
    # We run the while loop, unless we don't run out of the items in the frontier.
    # The problem cannot be solved if we don't find the solution and run out of items in the frontier.
    while not frontier.empty():
        # We get the next item in queue as store it
        node = frontier.get()
        
        # Prevents repetition if the node has already been pruned
        if node.pruned:
            continue
        
        # We break from the loop and return solution if the puzzle is solved
        if node.checkIfSolved():
            # g_score is the actual cost to reach the current node
            # If the puzzle is solved, this is the total number of steps
            steps = node.g_score
            break
        
        # This holds the blank/empty tile.
        coor_blank = tuple(*np.argwhere(np.array(node.puzzle_config)==0))
        
        # We can either move upwards, downwards, leftwards or rightwards.
        slides = [(0,1), (0, -1), (1,0), (-1,0)]
        
        # Based on the possible movements, we get all the valid coordinates for the successor nodes.
        all_slides = [tuple([sum(i) for i in zip(coor_blank, moves) if 0 <= sum(i) < dim]) for moves in slides]
        pos_slides = [i for i in all_slides if len(i) == 2]
        
        # From the list of all possible and valid slides.
        for m in pos_slides:
            # Making one more movement would increase the actual steps by 1.
            g_score = node.g_score + 1
            
            # Creates a temporary copy for the puzzle configuration
            temp_ = np.copy(node.puzzle_config)
            # Swaps the position between the blank and the other tile from the possible slides.
            temp_[coor_blank], temp_[m]=temp_[m], temp_[coor_blank]
            
            # If we already have the cost stored in dict_costs, we mark it for removal from the frontier.
            if tuple(temp_.flatten()) in dict_costs:
                # If the new found path is better than the previous one
                if dict_costs[tuple(temp_.flatten())].g_score > g_score:
                    dict_costs[tuple(temp_.flatten())].pruned = True
                # Else, we already have a better path and continue with other moves.
                else:
                    continue
            
            # h_score is the score that is returned by the respective heuristics.
            h_score = heuristic(temp_)
            
            # Child node will have the updated total cost (f_score) and other values.
            child_node = PuzzleNode(f_score = g_score + h_score, g_score = g_score, parent_node = node, puzzle_config = temp_)
            # We add the child node to the frontier.
            frontier.put(child_node)
            # Since we have explored the node, we store it as already explored.
            dict_costs[tuple(temp_.flatten())] = child_node
            # Add one to the number of explored nodes for return.
            exp += 1
    
    # Holds the solution for the problem, so it can be returned
    opt_ = []
    
    while node is not None:
        opt_.append(node)
        node = node.parent_node
    
    # Stores the optimal path and slides that we get from the algorithm.
    opt_path = []
    for i, n in enumerate(opt_[::-1]):
        opt_path.append(list(list(_) for _ in n.puzzle_config))
    
    max_frontier = frontier.qsize()
    err = 0
    return (steps, exp, max_frontier, opt_path, err)



In [4]:
## Test for state not correctly defined

incorrect_state = [[0,1,2],[2,3,4],[5,6,7]]
_,_,_,_,err = solvePuzzle(incorrect_state, lambda state: 0)
assert(err == -1)

In [5]:
## Heuristic function tests for misplaced tiles and manhattan distance

# Define the working initial states
working_initial_states_8_puzzle = ([[2,3,7],[1,8,0],[6,5,4]], [[7,0,8],[4,6,1],[5,3,2]], [[5,7,6],[2,4,3],[8,1,0]])

# Test the values returned by the heuristic functions
h_mt_vals = [7,8,7]
h_man_vals = [15,17,18]

for i in range(0,3):
    h_mt = heuristics[0](working_initial_states_8_puzzle[i])
    h_man = heuristics[1](working_initial_states_8_puzzle[i])
    assert(h_mt == h_mt_vals[i])
    assert(h_man == h_man_vals[i])


In [ ]:
## A* Tests for 3 x 3 boards
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

# Optimal path to the solution for the first 3 x 3 state
opt_path_soln = [[[2, 3, 7], [1, 8, 0], [6, 5, 4]], [[2, 3, 7], [1, 8, 4], [6, 5, 0]], 
                 [[2, 3, 7], [1, 8, 4], [6, 0, 5]], [[2, 3, 7], [1, 0, 4], [6, 8, 5]], 
                 [[2, 0, 7], [1, 3, 4], [6, 8, 5]], [[0, 2, 7], [1, 3, 4], [6, 8, 5]], 
                 [[1, 2, 7], [0, 3, 4], [6, 8, 5]], [[1, 2, 7], [3, 0, 4], [6, 8, 5]], 
                 [[1, 2, 7], [3, 4, 0], [6, 8, 5]], [[1, 2, 0], [3, 4, 7], [6, 8, 5]], 
                 [[1, 0, 2], [3, 4, 7], [6, 8, 5]], [[1, 4, 2], [3, 0, 7], [6, 8, 5]], 
                 [[1, 4, 2], [3, 7, 0], [6, 8, 5]], [[1, 4, 2], [3, 7, 5], [6, 8, 0]], 
                 [[1, 4, 2], [3, 7, 5], [6, 0, 8]], [[1, 4, 2], [3, 0, 5], [6, 7, 8]], 
                 [[1, 0, 2], [3, 4, 5], [6, 7, 8]], [[0, 1, 2], [3, 4, 5], [6, 7, 8]]]

astar_steps = [17, 25, 28]
for i in range(0,3):
    steps_mt, expansions_mt, _, opt_path_mt, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[0])
    steps_man, expansions_man, _, opt_path_man, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_mt == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
    assert(expansions_man < expansions_mt)
    # For the first state, test that the optimal path is the same
    if i == 0:
        assert(opt_path_mt == opt_path_soln)


In [7]:
## A* Test for 4 x 4 board
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

working_initial_state_15_puzzle = [[1,2,6,3],[0,9,5,7],[4,13,10,11],[8,12,14,15]]
steps_mt, expansions_mt, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[0])
steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[1])
# Test whether the number of optimal steps is correct and the same
assert(steps_mt == steps_man == 9)
# Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
assert(expansions_mt >= expansions_man)

In [8]:
## Puzzle solvability test

unsolvable_initial_state = [[7,5,6],[2,4,3],[8,1,0]]
_,_,_,_,err = solvePuzzle(unsolvable_initial_state, lambda state: 0)
assert(err == -2)